In [ ]:
import torch

if torch.cuda.is_available():
    gpu_id = torch.cuda.current_device()  # or whichever GPU ID you want to check
    props = torch.cuda.get_device_properties(gpu_id)
    total: float = props.total_memory / (1024**3)  # in GB
    reserved = torch.cuda.memory_reserved(gpu_id) / (1024**3)
    allocated = torch.cuda.memory_allocated(gpu_id) / (1024**3)
    free_torch = reserved - allocated  # what PyTorch thinks is free in its cache

    print(f"Current device: {torch.cuda.device(gpu_id)}")
    print(f"GPU name: {torch.cuda.get_device_name(gpu_id)}")
    print(f"Total GPU memory: {total:.2f} GB")
    print(f"Currently allocated by tensors: {allocated:.2f} GB")
    print(f"Reserved by PyTorch caching allocator: {reserved:.2f} GB")
    print(f"Free inside PyTorch cache: {free_torch:.2f} GB")
else:
    print("CUDA not available")


Current device: <torch.cuda.device object at 0x7f5594029c90>
GPU name: NVIDIA A100-PCIE-40GB
Total GPU memory: 39.56 GB
Currently allocated by tensors: 0.00 GB
Reserved by PyTorch caching allocator: 0.00 GB
Free inside PyTorch cache: 0.00 GB


In [8]:
!pwd

/eos/home-i01/a/ahojrup/SWAN_projects/Pytorch


In [11]:
!nvidia-smi

Mon Sep 15 12:16:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  |   00000000:00:06.0 Off |                    0 |
| N/A   35C    P0             39W /  250W |       3MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [12]:
def max_allocatable(gpu_id=0):
    torch.cuda.empty_cache()
    torch.cuda.set_device(gpu_id)
    left, right = 1, torch.cuda.get_device_properties(gpu_id).total_memory
    best = 0
    while left <= right:
        mid = (left + right) // 2
        try:
            tensor = torch.empty(mid // 4, dtype=torch.float32, device=f"cuda:{gpu_id}")
            del tensor
            torch.cuda.empty_cache()
            best = mid
            left = mid + 1
        except RuntimeError:
            right = mid - 1
    return best / (1024**3)

print(f"Max allocatable: {max_allocatable():.2f} GB")

Max allocatable: 39.14 GB
